In [8]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import requests
from nba_api.stats import endpoints
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

In [9]:
import DB_setup
mvp = pd.read_csv("./data/mvp_list.csv")

# mvps = DB_setup.mvp_data(mvp,0,10)
# mvps = pd.concat([mvps,DB_setup.mvp_data(mvp,10,20)])
# mvps = pd.concat([mvps,DB_setup.mvp_data(mvp,20,30)])
# mvps = pd.concat([mvps,DB_setup.mvp_data(mvp,30,40)])
# mvps = pd.concat([mvps,DB_setup.mvp_data(mvp,40,50)])
# mvps = pd.concat([mvps,DB_setup.mvp_data(mvp,50,60)])
# mvps = pd.concat([mvps,DB_setup.mvp_data(mvp,60,66)])




Uses multiple calls to the mvp_data setup function. This function matches the mvp list name and season to the NBA API pull and pulls from the league leaders list. (League leaders done to avoid having to pull to find player id numbers). This compiles a list of basic stats for a player during their mvp season. This is then converted to a csv, which is used for the rest of the notebook

In [10]:
stats = pd.read_csv("./data/mvp_stats.csv")

In [11]:
stats[["AGE","WS","WS/48","B2B","SEASON"]] = mvp[["Age","WS","WS/48","B2B","season"]]

stats.columns

Index(['Unnamed: 0', 'PLAYER_ID', 'RANK', 'PLAYER', 'TEAM', 'GP', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'EFF',
       'AST_TOV', 'STL_TOV', 'AGE', 'WS', 'WS/48', 'B2B', 'SEASON'],
      dtype='object')

In [12]:
import calc_stats

stats["EF%"] = calc_stats.eff_fga(stats["FGM"],stats["FG3M"],stats["FGA"])
stats["TS%"] = calc_stats.ts_pct(stats["PTS"],stats["FGA"],stats["FTA"])
stats["FTR"] = calc_stats.ft_rate(stats["FTA"],stats["FGA"])
stats["HAR"] = calc_stats.holl_ar(stats["AST"],stats["FGA"],stats["FTA"],stats["TOV"])
stats["TOR"] = calc_stats.tov_pct(stats["AST"],stats["FGA"],stats["FTA"],stats["TOV"])

stats.head(10)

,Unnamed: 0,PLAYER_ID,RANK,PLAYER,TEAM,GP,MIN,FGM,FGA,FG_PCT,...,AGE,WS,WS/48,B2B,SEASON,EF%,TS%,FTR,HAR,TOR
0,2,203999,3,Nikola Jokic,DEN,72,2488,732,1293,0.566,...,25,15.6,0.301,0,2020,0.601701,0.641151,0.304718,0.260305,0.096474
1,3,203507,4,Giannis Antetokounmpo,MIL,63,1917,685,1238,0.553,...,25,11.1,0.279,1,2019,0.589257,0.604187,0.508078,0.166920,0.108451
2,6,203507,7,Giannis Antetokounmpo,MIL,72,2358,721,1247,0.578,...,24,14.4,0.292,0,2018,0.599038,0.633881,0.550120,0.187209,0.118330
3,1,201935,2,James Harden,HOU,72,2551,651,1449,0.449,...,28,15.4,0.289,0,2017,0.540718,0.610536,0.501725,0.229984,0.114992
4,0,201566,1,Russell Westbrook,OKC,81,2802,824,1941,0.425,...,28,13.1,0.224,0,2016,0.476043,0.546581,0.432767,0.232172,0.121061
5,1,201939,2,Stephen Curry,GSW,79,2700,805,1598,0.504,...,27,17.9,0.318,1,2015,0.629537,0.664150,0.250313,0.204501,0.101669
6,1,201939,2,Stephen Curry,GSW,80,2613,653,1341,0.487,...,26,15.7,0.288,0,2014,0.593587,0.632880,0.251305,0.261283,0.105104
7,0,201142,1,Kevin Durant,OKC,81,3122,849,1688,0.503,...,25,19.2,0.295,0,2013,0.559834,0.626215,0.476896,0.158907,0.101772
8,2,2544,3,LeBron James,MIA,76,2877,765,1354,0.565,...,28,19.3,0.322,1,2012,0.603028,0.633035,0.395126,0.231015,0.094754
9,1,2544,2,LeBron James,MIA,62,2326,621,1169,0.531,...,27,14.5,0.298,0,2011,0.554320,0.597890,0.429427,0.192782,0.106105


In [13]:

stats["PTS"] = calc_stats.per_game(stats["PTS"],stats["GP"])
stats["MIN"] = calc_stats.per_game(stats["MIN"],stats["GP"])
stats["FGM"] = calc_stats.per_game(stats["FGM"],stats["GP"])
stats["FGA"] = calc_stats.per_game(stats["FGA"],stats["GP"])
stats["FG3M"] = calc_stats.per_game(stats["FG3A"],stats["GP"])
stats["FTM"] = calc_stats.per_game(stats["FTM"],stats["GP"])
stats["FTA"] = calc_stats.per_game(stats["FTA"],stats["GP"])
stats["OREB"] = calc_stats.per_game(stats["OREB"],stats["GP"])
stats["DREB"] = calc_stats.per_game(stats["DREB"],stats["GP"])
stats["REB"] = calc_stats.per_game(stats["REB"],stats["GP"])
stats["AST"] = calc_stats.per_game(stats["AST"],stats["GP"])
stats["STL"] = calc_stats.per_game(stats["STL"],stats["GP"])
stats["BLK"] = calc_stats.per_game(stats["BLK"],stats["GP"])
stats["TOV"] = calc_stats.per_game(stats["TOV"],stats["GP"])
stats["PF"] = calc_stats.per_game(stats["PF"],stats["GP"])

The last two boxes calculate new sets of stats. The first set is all advanced stats, the next is every counting stat converted to a per game stat.

In [14]:
team_df =  endpoints.leaguedashptteamdefend.LeagueDashPtTeamDefend().get_data_frames()[0]
team_ids = team_df["TEAM_ID"]

team = DB_setup.team_data(team_ids)

/Users/mickflannery/Documents/GitHub/NBA_stats/DB_setup.py:23: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  data = pd.concat(data,team)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"